# <h3><b>Library

In [163]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import subprocess
import json
import sys
from langchain_openai import AzureChatOpenAI
from openai import OpenAI
from langchain.agents import initialize_agent
from langchain_community.tools import Tool, DuckDuckGoSearchRun, DuckDuckGoSearchResults
from ddgs import DDGS
from langchain.agents import Tool
from langchain.agents import initialize_agent
import os
from langchain.tools import BaseTool
import requests
from bs4 import BeautifulSoup
import requests
from groq import Groq
import os
import json
import subprocess
from urllib.parse import urlencode
import urllib.request
import xml.etree.ElementTree as ET

# <h3><b>Class Tools

In [ ]:
class WebScraperTool(BaseTool):
    name: str= "Web Scraper"
    description: str = "Fetches the full content of a webpage given its URL."

    def _run(self, url: str) -> str: 
        try:
            url = f'https://r.jina.ai/{url}'
            response = requests.get(url)
            html_content = response.text

            soup = BeautifulSoup(html_content, 'html.parser')
            response = soup.prettify()
            return response
        except Exception as e:
            return f"Error: {e}"

web_scraper = WebScraperTool()

class DDGSSearchTool(BaseTool):
    name: str = "DuckDuckGo Search"
    description: str = "Digunakan untuk searching di web tapi bukan membuka web."
    
    def _run(self, query: str) -> str:
        try:
            with DDGS() as ddgs:
                results = list(ddgs.text(query, max_results=5))
                formatted_results = []
                for result in results:
                    formatted_results.append(f"Title: {result['title']}\nURL: {result['href']}\nSnippet: {result['body']}\n")
                return "\n".join(formatted_results)
        except Exception as e:
            return f"Error: {e}"

# Ganti bagian ini:
ddg_search = DDGSSearchTool()

class ArxivAPI(BaseTool):
    name: str = "Arxiv API"
    description: str = "Digunakan untuk mencari paper paper yang mendukung untuk riset aplikasi. Inputannya adalah keyword pencarian."
    
    def _run(self, query: str) -> str:
        try:
            # Fungsi untuk mengambil dan mem-parsing data XML dari arXiv dengan search_query dan rentang tahun
            def fetch_arxiv_with_query_and_date_range(query, start_year, end_year):
                start_date = f"{start_year}0101000000"  # 1 Januari di awal tahun
                end_date = f"{end_year}1231235959"     # 31 Desember di akhir tahun

                # Gabungkan search_query dengan rentang tahun menggunakan operator AND
                search_query = f"ti:{query} AND submittedDate:[{start_date} TO {end_date}]"

                params = {
                    "search_query": search_query,
                    "sortBy": "submittedDate",  # Mengurutkan berdasarkan tanggal submission
                    "sortOrder": "ascending"
                }
                base_url = "http://export.arxiv.org/api/query"
                url = f"{base_url}?{urlencode(params)}"

                # Mengambil data dari URL
                with urllib.request.urlopen(url) as response:
                    data = response.read().decode('utf-8')

                # Parsing XML
                root = ET.fromstring(data)

                # Namespace arXiv
                namespaces = {'atom': 'http://www.w3.org/2005/Atom'}

                # Mencari semua entri (artikel)
                entries = root.findall('atom:entry', namespaces)

                # Iterasi setiap entri dan cetak informasi penting
                for i, entry in enumerate(entries, start=1):
                    title = entry.find('atom:title', namespaces).text  # Judul artikel
                    authors = [author.find('atom:name', namespaces).text for author in entry.findall('atom:author', namespaces)]  # Daftar penulis
                    published_date = entry.find('atom:published', namespaces).text  # Tanggal publikasi
                    summary = entry.find('atom:summary', namespaces).text  # Abstrak
                    link = entry.find('atom:id', namespaces).text  # Link ke artikel
                    
                    print(f"Artikel {i}:")
                    print(f"Judul: {title}")
                    print(f"Penulis: {', '.join(authors)}")
                    print(f"Tanggal Publikasi: {published_date}")
                    print(f"Abstrak: {summary[:200]}...")  # Cetak 200 karakter pertama dari abstrak
                    print(f"Link: {link}")
                    print("-" * 80)

            # Panggil fungsi utama dengan query "electron thermal conductivity" dan rentang tahun 2010 hingga 2020
            return str(fetch_arxiv_with_query_and_date_range(query, 2023, 2025))

        except Exception as e:
            return f"Error: {e}"

# Ganti bagian ini:
api_arxiv = ArxivAPI()

class TelegramAPI(BaseTool):
    name: str = "Telegram API"
    description: str = "Digunakan untuk memberikan 'message' bahwa proses riset sudah selesai dan memberikan hasil keseluruhan."
    
    def _run(self, message: str) -> str:
        try:
            # Fungsi untuk mengirim pesan ke Telegram
            def send_telegram_message(api_token, chat_id, message):
                url = f"https://api.telegram.org/bot{api_token}/sendMessage"
                payload = {
                    "chat_id": chat_id,
                    "text": message
                }
                response = requests.post(url, data=payload)
                if response.status_code == 200:
                    print("Pesan berhasil dikirim!")
                else:
                    print(f"Gagal mengirim pesan. Status code: {response.status_code}")
                    print(f"Response: {response.text}")

            # Konfigurasi
            api_token = "api_token"  # Ganti dengan token bot Anda
            chat_id = "chat_id"           # Ganti dengan Chat ID penerima

            # Kirim pesan
            send_telegram_message(api_token, chat_id, message)
            return "Mengirim Pesan Selesai!"
        except Exception as e:
            return f"Error: {e}"

# Ganti bagian ini:
api_telegram = TelegramAPI()

# <h3><b>Tools

In [ ]:
tools = [
    Tool(
        name="DuckDuckGo Search",
        func=ddg_search.run,
        description="Digunakan untuk searching di web tapi bukan membuka web.",
    ),
    Tool(
        name="Web Scraper",
        func=web_scraper.run,
        description="Mendapatkan semua isian suatu web tapi bukan yang membutuhkan login seperti sosial media. Masukkan yang dibutuhkan adalah link url yang dimulai dengan http atau https.",
    ),
    Tool(
        name="Arxiv API",
        func=api_arxiv.run,
        description="Digunakan untuk mencari paper paper yang mendukung untuk riset aplikasi. Inputannya adalah keyword pencarian."
    )
]

tool_telegram = [
    Tool(
        name="Telegram API",
        func=api_telegram.run,
        description="Digunakan untuk memberikan 'message' bahwa proses riset sudah selesai dan memberikan hasil keseluruhan."
    )
]

In [ ]:
os.environ["AZURE_OPENAI_API_KEY"] = "key"
os.environ["AZURE_OPENAI_ENDPOINT"] = "endpoint"

llm = AzureChatOpenAI(
    deployment_name="gpt-4.1",  
    model_name="gpt-4.1",      
    temperature=0,
    api_version="2025-01-01-preview"
)

agent_riset = initialize_agent(
    tools, llm, agent="zero-shot-react-description", verbose=True, handle_parsing_errors=True
)

agent_telegram = initialize_agent(
    tool_telegram, llm, agent="zero-shot-react-description", verbose=True, handle_parsing_errors=True
)

# <h3><b>Prompt

In [167]:
def prompt_markdown(user_input):
    prompt = f"""## Persona
Anda adalah seorang Product Research Analyst AI yang berpengalaman dalam riset pasar teknologi dan analisis kompetitif. Anda memiliki keahlian dalam:
- Menganalisis tren pasar aplikasi digital dan teknologi
- Mengidentifikasi kompetitor dan peluang diferensiasi
- Mengevaluasi fitur-fitur inovatif dalam aplikasi
- Menyintesis temuan akademis untuk aplikasi praktis
- Memberikan rekomendasi strategis yang actionable

## Instructions
Lakukan riset komprehensif terhadap aplikasi yang disebutkan user dengan langkah-langkah berikut:

1. **Analisis Pasar & Kompetitor**
   - Gunakan web scraping untuk mencari aplikasi serupa yang sudah ada
   - Identifikasi minimal 2-7 kompetitor utama
   - Analisis positioning dan value proposition masing-masing kompetitor
   - Evaluasi kelebihan dan kekurangan kompetitor

2. **Research Paper Akademis** ⚠️ PENTING - ATURAN ARXIV
   - **LANGKAH WAJIB**: Gunakan tool arXiv API dengan multiple kata kunci berbeda
   - **PROSES HASIL**: Jika tool arXiv memberikan hasil (entries/papers), WAJIB gunakan semua data yang diberikan
   - **JANGAN ABAIKAN**: Jika tool arXiv menampilkan paper dalam responsenya, itu berarti paper ADA dan VALID
   - **FORMAT DATA**: Gunakan PERSIS data yang dikembalikan tool arXiv:
     - Title: ambil dari field 'title' 
     - Authors: ambil dari field 'authors'
     - Published: ambil dari field 'published'
     - ID: ambil dari field 'id' 
     - Summary: ambil dari field 'summary'
     - Link: format sebagai https://arxiv.org/abs/[ID]
   - **JANGAN TULIS "TIDAK DITEMUKAN"** jika tool arXiv sudah memberikan hasil
   - **VALIDASI**: Hanya tulis "tidak ditemukan" jika tool arXiv benar-benar return kosong/error
   - Cari dengan kata kunci: teknologi utama aplikasi, metodologi terkait, domain aplikasi

3. **Validasi Hasil ArXiv** 🔍
   - Sebelum menuliskan paper dalam output final, **SELALU cek ulang**:
     - Apakah judul paper PERSIS sama dengan hasil API?
     - Apakah ID paper benar dan sesuai dengan hasil API?
     - Apakah abstrak sesuai dengan yang diberikan API?
   - Jika ada ketidaksesuaian, JANGAN sertakan paper tersebut

4. **Analisis Gap & Opportunity**
   - Identifikasi celah pasar yang belum terisi oleh kompetitor
   - Analisis pain points yang belum teratasi
   - Evaluasi tren teknologi yang dapat dimanfaatkan

5. **Rekomendasi Fitur Diferensiasi**
   - Berikan 2-3 saran fitur unik yang dapat membedakan aplikasi
   - Prioritaskan berdasarkan feasibility dan market impact
   - Sertakan reasoning mengapa fitur tersebut penting

## Output Format
Berikan analisis dalam format markdown berikut:

# Analisis Riset Aplikasi: [Nama Aplikasi]

## 🎯 Executive Summary
*Ringkasan eksekutif dalam 3-4 kalimat mengenai temuan utama dan rekomendasi strategis*

## 📊 Analisis Kompetitor

### Kompetitor Utama
1. **[Nama Aplikasi]**
   - **Value Proposition**: [Proposisi nilai utama]
   - **Kelebihan**: [2-3 kelebihan utama]
   - **Kekurangan**: [2-3 kelemahan]
   - **Market Position**: [Posisi di pasar]

2. **[Nama Aplikasi]**
   - **Value Proposition**: [Proposisi nilai utama]
   - **Kelebihan**: [2-3 kelebihan utama]
   - **Kekurangan**: [2-3 kelemahan]
   - **Market Position**: [Posisi di pasar]

*[Lanjutkan untuk kompetitor lainnya]*

### Insight Kompetitor
- **Tren Dominan**: [Jelaskan pola umum yang ditemukan]
- **Gap Pasar**: [Identifikasi celah yang belum terisi]
- **Positioning Opportunity**: [Peluang posisi unik di pasar]

## 🚀 Rekomendasi Fitur Diferensiasi

### Fitur Prioritas Tinggi
1. **[Nama Fitur]**
   - **Deskripsi**: [Penjelasan detail fitur]
   - **Value Add**: [Nilai tambah spesifik yang diberikan]
   - **Differentiator**: [Mengapa unik dibanding kompetitor]
   - **Implementation Priority**: ⭐⭐⭐⭐⭐
   - **Technical Feasibility**: [Tingkat kemudahan implementasi]

2. **[Nama Fitur]**
   - **Deskripsi**: [Penjelasan detail fitur]
   - **Value Add**: [Nilai tambah spesifik yang diberikan]
   - **Differentiator**: [Mengapa unik dibanding kompetitor]
   - **Implementation Priority**: ⭐⭐⭐⭐
   - **Technical Feasibility**: [Tingkat kemudahan implementasi]

### Fitur Prioritas Menengah
*[Lanjutkan format yang sama untuk 2-3 fitur lainnya]*

## 🔬 Dukungan Riset Akademis

### Status Pencarian ArXiv
**Kata kunci yang digunakan**: [Daftar kata kunci yang dicoba]
**Tool arXiv Response**: [Ringkasan response dari tools - berapa paper ditemukan]
**Status hasil**: [Berhasil menemukan X paper dari tool arXiv]

### Paper Rekomendasi Utama
**⚠️ SUMBER**: Semua paper di bawah ini diambil LANGSUNG dari response tool arXiv API.

1. **Paper 1**
   - **Judul**: [Copy PERSIS dari field 'title' tool arXiv]
   - **Penulis**: [Copy dari field 'authors' tool arXiv]
   - **Tanggal Publikasi**: [Copy dari field 'published' tool arXiv]
   - **ArXiv ID**: [Copy dari field 'id' tool arXiv]
   - **Link**: https://arxiv.org/abs/[ID dari field 'id']
   - **Abstrak**: [Copy dari field 'summary' tool arXiv - maksimal 3 kalimat pertama]
   - **Relevansi**: [Analisis mengapa relevan untuk aplikasi yang diriset]

2. **Paper 2**
   - **Judul**: [Copy PERSIS dari field 'title' tool arXiv]
   - **Penulis**: [Copy dari field 'authors' tool arXiv]
   - **Tanggal Publikasi**: [Copy dari field 'published' tool arXiv]
   - **ArXiv ID**: [Copy dari field 'id' tool arXiv]
   - **Link**: https://arxiv.org/abs/[ID dari field 'id']
   - **Abstrak**: [Copy dari field 'summary' tool arXiv - maksimal 3 kalimat pertama]
   - **Relevansi**: [Analisis mengapa relevan untuk aplikasi yang diriset]

*[Lanjutkan untuk semua paper yang dikembalikan tool arXiv, maksimal 5 paper terbaru]*

**INSTRUKSI KHUSUS UNTUK AGENT:**
- Jika tool arXiv memberikan response dengan paper (ada field title, authors, dll), WAJIB tulis semua paper tersebut
- COPY PASTE langsung data dari response tool tanpa mengubah apapun
- JANGAN menulis "tidak ditemukan" jika tool sudah memberikan hasil
- Hanya tulis "tidak ditemukan" jika response tool benar-benar kosong atau error

**JIKA TOOL ARXIV TIDAK MEMBERIKAN HASIL APAPUN (response kosong/error), BARU TULIS:**
```
### Status Pencarian ArXiv
**Kata kunci yang digunakan**: [kata kunci 1], [kata kunci 2], [kata kunci 3]
**Tool arXiv Response**: Tidak ada response atau error dari tool
**Status hasil**: Tool arXiv tidak memberikan hasil untuk kata kunci yang dicoba

### Rekomendasi Alternatif
Untuk mendapatkan insight akademis, disarankan untuk:
1. Mencari di Google Scholar dengan kata kunci yang sama
2. Mengecek konferensi teknologi terkait
3. Mencari white paper dari perusahaan teknologi terkemuka
```

**PENTING**: Jangan gunakan template di atas jika tool arXiv sudah memberikan paper dalam responsenya!

### Trend Teknologi dari Literature
*[Hanya isi bagian ini jika ada paper yang ditemukan]*
- **[Trend 1]**: [Penjelasan berdasarkan paper yang ditemukan]
- **[Trend 2]**: [Penjelasan berdasarkan paper yang ditemukan]
- **[Trend 3]**: [Penjelasan berdasarkan paper yang ditemukan]

### Technical Considerations
- **Core Technologies**: [Teknologi inti yang direkomendasikan]
- **Architecture Suggestions**: [Saran arsitektur sistem]
- **Scalability Factors**: [Faktor-faktor untuk pertimbangan scaling]
- **Implementation Roadmap**: [Tahapan implementasi yang disarankan]

---
*Riset dilakukan pada: [Tanggal] | Tools used: Web Search, ArXiv API*

## User Question
{user_input}
"""
    return prompt

In [ ]:
prompt = prompt_markdown("Saya sedang mengembangkan aplikasi bernama Snailly. Aplikasi ini ditujukkan untuk membuat internet anak menjadi aman. Saat ini terdapat fitur yaitu Surfior untuk memblokir konten negatif, notifior untuk memberikan notif ke orang tua jika anak mengakses konten negatif, dan reportior untuk memberikan report history anak ke orang tua.")
# prompt = prompt_markdown("Saya sedang riset terkait federated learning. Ini digunakan untuk membuat model klasifikasi personalized bagi user.")
hasil_riset = agent_riset.run(prompt)
kirim_chat = agent_telegram.run(f"Kirimkan hasil dari Agen Riset ke Chat Telegram. Kirimkan semua hasilnya tanpa terpotong sedikitpun dengan dipotong menjadi 3 bagian, setelah dipotong lalu mengirim masing masing bagian, jadi total mengirim sebanyak 3 kali. Hal ini dilakukan untuk menghindari message terlalu panjang!!!! Berikut hasil risetnya: {hasil_riset}")




> Entering new AgentExecutor chain...
Thought: Untuk riset komprehensif aplikasi Snailly, saya perlu:
1. Mengidentifikasi dan menganalisis kompetitor utama di ranah parental control, internet safety, dan monitoring anak.
2. Melakukan pencarian paper akademis terkait parental control, internet filtering, child online safety, dan monitoring apps.
3. Menganalisis gap pasar, tren teknologi, dan peluang diferensiasi fitur.
4. Memberikan rekomendasi fitur inovatif berbasis temuan pasar dan akademis.

Langkah pertama: identifikasi kompetitor utama dan analisis positioning serta value proposition mereka.

Action: Web Scraper
Action Input: https://play.google.com/store/search?q=parental%20control%20internet%20safety%20kids&c=apps
Observation: Title: parental control internet safety kids - Android Apps on Google Play

URL Source: https://play.google.com/store/search?q=parental%20control%20internet%20safety%20kids&amp;c=apps

Markdown Content:
parental control internet safety kids - Android App